# Lesson 4 - Compare with P.2108 Clutter Model

In this lesson you'll compare the performance of your clutter propagation model with the P.2108 clutter model.

### About P.2108
 * __The documentation__ for the P.2108 clutter model can be found at [Recommendation ITU-R P.2108-1](https://www.itu.int/dms_pubrec/itu-r/rec/p/R-REC-P.2108-1-202109-I!!PDF-E.pdf).
 * __The code repository__ for P.2108 can be found at [NTIA/p2108](https://github.com/NTIA/p2108) github repository.

The P.2108 recommendation contains three methods for predicting clutter loss:
1. Height Gain Terminal Correction Model, for 0.3 to 3 GHz
2. Terrestrial Statistical Model, for 2 to 67 GHz
3. Aeronautical Statistical Model, for 10 to 100 GHz

We will use the Terrestrail Statistical Model because our measurement data is at 3.5 GHz.

### Import the P.2108 code library
The NTIA code repository for P.2108 contains the U.S. reference implemention for all three P.2108 clutter loss prediction methods listed above. In **`course-materials/packages/p2108-1.0.0-py3-none-any.whl`** we've included an installable Python package for the P.2108 software. 

Execute the following cell to install the P.2108 package in your JupyterLab environment.

In [ ]:
! pip install packages/p2108-1.0.0-py3-none-any.whl

### Import the necessary python libraries

In [ ]:
from ITS.ITU.PSeries import P2108
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### How to use p2108

In [ ]:
L__dB = P2108.TerrestrialStatisticalModel(3.5, 0.95, 50)

In [ ]:
print(L__dB)

In [ ]:
greater_p25_df = pd.read_csv("data/MartinAcres_Lesson4.csv")

In [ ]:
slope = 13.71
y_int = -9.8
model_std = 3.8
## define the clutter model
def model(r_c):
    return slope * np.log10(r_c) + y_int

In [ ]:
greater_p25_df["p2108"] = greater_p25_df.apply(lambda row: P2108.TerrestrialStatisticalModel(row.f__mhz/1000, row.d__km, 50), axis=1)


In [ ]:
greater_p25_df.info()

In [ ]:
plt.scatter(greater_p25_df["d__km"], greater_p25_df["L_excess__db"], label='Measurements', s=12)
plt.scatter(greater_p25_df["d__km"], greater_p25_df["p2108"], label='p2108 predictions', s=12)
plt.scatter(greater_p25_df["d__km"], greater_p25_df["pred_loss"], label='EuCAP Model predictions', s=12)

plt.xlabel('Distance (m)')
plt.ylabel('Clutter Loss (dB)')
plt.title('Path Distance vs Clutter Loss')

plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

In [ ]:
greater_p25_df.head(5)

In [ ]:
distances_array = np.sort(greater_p25_df["d__km"])
clutter_distances_array = np.sort(greater_p25_df["clutter_d__meter"])

In [ ]:
p2108_cdf_distri = []
for d in distances_array:
    p2108_cdf_distri.append(P2108.TerrestrialStatisticalModel(greater_p25_df["f__mhz"][0]/1000, d, np.random.randint(1,100)))

model_cdf_distri = []
for d in clutter_distances_array:
    model_cdf_distri.append(np.random.normal(model(d), model_std))

In [ ]:
plt.rcParams["figure.figsize"] = (11,6)

meas_N = len(greater_p25_df)
meas_x = np.sort(greater_p25_df["L_excess__db"])
meas_y = np.arange(meas_N) / float(meas_N)
## plot the CDF
plt.plot(meas_x, meas_y, label='Meas')

p2108_x = np.sort(np.array(p2108_cdf_distri))
p2108_y = np.arange(meas_N) / float(meas_N)
## plot the CDF
plt.plot(p2108_x, p2108_y, label='p2108')

model_x = np.sort(np.array(model_cdf_distri))
model_y = np.arange(meas_N) / float(meas_N)
## plot the CDF
plt.plot(model_x, model_y, label='Model')

plt.xlabel('Clutter Loss (dB)')
plt.ylabel('Probability')
plt.title('Clutter Loss CDF')

plt.gca().yaxis.grid(True)
plt.legend(fontsize=14)
plt.show()